# make korad txt file (coco format)

In [6]:
import json
import os
import glob
import random
import numpy as np
import tqdm
import shutil

### VOC box (xmin,ymin,xmax,ymax) to COCO box (xmin,ymin, box-width(xmax-xmin), box-height(ymax-ymin))

In [2]:
def convert(size, box):
    dw = 1./(size[0])
    dh = 1./(size[1])
    x = (box[0] + box[1])/2.0 - 1
    y = (box[2] + box[3])/2.0 - 1
    w = box[1] - box[0]
    h = box[3] - box[2]
    x = '{:.6f}'.format(round(x*dw,6))
    w = '{:.6f}'.format(round(w*dw,6))
    y = '{:.6f}'.format(round(y*dh,6))
    h = '{:.6f}'.format(round(h*dh,6))
    if np.float16(x) < 0 : x = "0.000000"
    if np.float16(y) < 0 : y = "0.000000"    
    return x,y,w,h

### path random 으로 섞기

In [11]:
np.random.seed(2)

root_path = os.path.expanduser("~/Desktop/joo/data/korad/data/custom/")
label_set = set(map(lambda x : x[-13:-4] if x[-4:]==".jpg" else x[-14:-5],glob.glob(root_path+"images_plan/*/*")))
labels = list(label_set)
random.shuffle(labels)

In [12]:
with open("./korad_boxes/all_list.json", "r") as f:
    all_dict = json.load(f)
    train_ratio = 0.82
    val_ratio = 0.15
    test_ratio = 0.03

    train_set = 0
    val_set = 0
    test_set = 0
    
    error_set = 0

    for index, data_id in enumerate(all_dict.keys()):
        # if index == 10 : break
            
        rd_num = np.random.random()
        
        _, _, json_true = all_dict[data_id]
        
        if json_true == None :rd_num = 1
        
        if rd_num < train_ratio :
            train_set += 1
            datatype = ["train"]

        elif rd_num < train_ratio + val_ratio:
            val_set += 1
            datatype = ["val"]
        else :
            if rd_num == 1 : error_set += 1
            test_set += 1
            datatype = ["test"]
        all_dict[data_id] = all_dict[data_id] + datatype
 

In [13]:
print(f"train_set : {train_set}개 ({train_set/23400:.2f}%)")
print(f"val_set : {val_set}개 ({val_set/23400:.2f}%)")
print(f"test_set : {test_set - error_set} + {error_set}(error file) = {test_set}개 ({test_set/23400:.2f}%)")

train_set : 18421개 (0.79%)
val_set : 3419개 (0.15%)
test_set : 646 + 914(error file) = 1560개 (0.07%)


In [15]:
classes = ["ConcreteCrack", "AlligatorCrack","Spalling", "Efflorescene", "Exposure"]
year = "2022"
save_path = ["train","val","test"]
save_type = ["2022","2023"]
save_type_num = 0 #0 == multi  /  1 == single
error_label = []

# txt 초기화
# for write_type in save_path:
#     with open(root_path+write_type+save_type[save_type_num]+".txt", "w") as f: f.writelines("")

# txt in labels
for file in labels:
    
    number, img_file, json_file_name, data_type = all_dict[file]

    # with open(root_path+data_type+save_type[save_type_num]+".txt", "a") as f: 
    #         f.writelines("./images/"+data_type+save_type[0]+"/"+img_file+"\n")
            # plan
            # file.writelines("./images/"+number+"/"+img_file+"\n")

    # train test val image 나누기
    shutil.copyfile(root_path+"images_plan/"+number+"/"+img_file, root_path+"images/"+data_type+save_type[save_type_num]+"/"+img_file)
    continue
    if json_file_name == None: continue
    
    with open(root_path+"labeling/"+number+"/"+json_file_name, "r") as f: json_file = json.load(f)

    width = json_file['images']['width']
    height = json_file['images']['height']
    save_true = True

    for label in json_file['labels']:
        class_num = str(classes.index(label['label']))
        xmin = label['boundingPoly']['vertices'][0]['xmin']
        ymin = label['boundingPoly']['vertices'][0]['ymin']
        xmax = label['boundingPoly']['vertices'][0]['xmax']
        ymax = label['boundingPoly']['vertices'][0]['ymax']

        bb = convert((width,height),(xmin,xmax,ymin,ymax))
        
        # concreteCrack 하나만 잡기
        if save_type_num != 0 and class_num != "0" : continue

        # error_label 제거
        if bb[0] == "0.000000" and bb[1] == "0.000000" and bb[2] == "0.000000" and bb[3] == "0.000000": 
            error_label.append({file:[label['label'],bb]})
            continue
        
        if save_true:
            with open(root_path+"labels/"+data_type+save_type[save_type_num]+"/"+json_file_name[:-5]+".txt", "w") as f:
                f.writelines(class_num + " "+bb[0]+" "+bb[1]+" "+bb[2]+" "+bb[3]+" \n")
            save_true = False
        else:
            with open(root_path+"labels/"+data_type+save_type[save_type_num]+"/"+json_file_name[:-5]+".txt", "a") as f:
                f.writelines(class_num + " "+bb[0]+" "+bb[1]+" "+bb[2]+" "+bb[3]+" \n")
    # break

In [67]:
print(f"제거된 error_label 개수 : {len(error_label)}개")
print(error_label)

제거된 error_label 개수 : 4개
[{'C03_00586': ['AlligatorCrack', ('0.000000', '0.000000', '0.000000', '0.000000')]}, {'C02_02018': ['Spalling', ('0.000000', '0.000000', '0.000000', '0.000000')]}, {'C04_03388': ['ConcreteCrack', ('0.000000', '0.000000', '0.000000', '0.000000')]}, {'C03_00193': ['Efflorescene', ('0.000000', '0.000000', '0.000000', '0.000000')]}]


loading Roboflow workspace...
loading Roboflow project...


Extracting Dataset Version Zip to Hard-Hat-Sample-2 in yolov5pytorch:: 100%|██████████| 491/491 [00:00<00:00, 10866.99it/s]
